## TOOL UPDATED: December 3 2024, Henrik Loecke

##Double click this cell to see full description.

##You must restart the kernel after updating Calibration_Variables.py!

<!-- 

To run this notebook, click menu Cell -> Run All

User input has been moved away from this notebook so it can easily be replaced by new versions.
 
Please open Calibration_Variables.py, in the same folder as this notebook, to edit user input there.

All variables with path must start with 'r', e.g. r'C:\Projects'

It must contain the following variables:

model_area:                          Short area name like 'VSA' or LISA'
generate_confidence_csvs:            Generate csv files for confidence maps, True/False
result_specs_csv:                    CSV file linking network and runoff result file. Only needed if runoff imported.
map_point_spacing:                   Space between dots in confidence maps, e.g. 100 (in meter but number withouth unit)
use_accumulation:                    Use for models with proper tree structure, all models except VSA, True/False
slope_source_unit_meter_per_meter    This is the case for NSSA and FSA, in VSA it is per thousand
model_area_strict_match:             If True, accept 'VSA' but not 'VSA-2019'. If False, accept both.
output_folder:                       Folder path where reports are to be created.
result_folder:                       Folder path of result files.
calibration_sheet:                   Full path, folder included, of calibration parameter sheet.
model:                               Full path, folder included, of model database.
summation_csv:                       Full path, folder included, of summation.csv.
node_csv:                            Full path, folder included, of MH_Zones.csv.
outfall_csv:                         Full path, folder included, of Outfall_Summary.csv.
rainfall_dfs0_file:                  Full path, folder included, of rainfall dfs0 file.
map_folder:                          Folder path where report maps are located.
dfs0_folders:                        Python list of all folders holding measurement dfs0 files.
 -->


In [ ]:
#PERMANENT CELL 1

import os
import re
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import numpy as np
import pandas as pd
import datetime as dt
import pickle
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import clear_output
import math
import sqlite3
from Calibration_Variables import *
import copy
import ctypes
import traceback
import shutil
import subprocess
MessageBox = ctypes.windll.user32.MessageBoxA


In [ ]:
#PERMANENT CELL 2
#The external script Read_Parameters.py is called to read data from the model.
try:
    
    
    if not (model.lower().endswith('.sqlite') or model.lower().endswith('.mdb')):
        raise ValueError("The variable 'model' must have .mdb or .sqlite extension: " + model)
    if os.path.exists(model) == False:
        raise ValueError("The variable 'model' points to a path that does not exist: " + model)    
    
    parameter_script = r"Read_Parameters.py"
    bat_file_path = 'Read_Parameters.bat'
    bat_file = open(bat_file_path, "w")
    bat_file.write(python_installation + ' "' + parameter_script + '" "' + os.getcwd() + '" "' + model + '" ' + str(use_accumulation))
    bat_file.close()
    result = subprocess.call([bat_file_path]) 

    if result == 1: #Error
        raise ValueError("The sub process threw an error. Please Locate the bat file: " + bat_file_path + ", open it in notepad, \
        then add a new line and type in letters only: Pause. Double click the bat file to run it and it will show the error.")
           
except Exception as e:    
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 2\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")


In [ ]:
#PERMANENT CELL 3
#Import csv files and set up different lists used later in the notebook.

try:
    dwf_csv = r"DWF_Specs.csv"
    res_types = ['Mixed','ResHD','ResLD']
    ici_types = ['Commercial','Industrial','Institutional']
    ww_types = res_types + ici_types

    all_types = []
    for res_type in res_types:
        all_types.append([res_type,'Population','pe','m3/pe/d'])
    for ici_type in ici_types:
        all_types.append([ici_type,'Area','ha','m3/ha/d'])
    all_types.append(['Baseflow','','',''])
    all_types.append(['Total','','',''])
    all_types = pd.DataFrame(all_types, columns =['Load_Type','Description','Unit1','Unit2'])
    all_types.set_index('Load_Type',inplace=True)

    period_specs = []
    period_specs.append('DWF')
    for i in range(1,7):
        period_specs.append('WWF' + str(i))
    period_specs

    dwf_specs = pd.read_csv(os.getcwd() + '\\' + dwf_csv)
    dwf_specs.set_index('Zone',inplace=True)
    
    network_specs = pd.read_csv(os.getcwd() + '\\Network.csv')
    network_specs.set_index('MUID',inplace=True)

    wwf_csv = r"WWF_Specs.csv"

    #used to maintain consistent color order for overflows
    colors = ['black','purple','green','orange','grey','brown']

    gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
    gauges = gauges.loc[:, :'Shift Y (m)']
    if model_area_strict_match == True:
        gauges = gauges[gauges.Model==model_area]
    else:
        gauges = gauges[gauges.Model.str.contains(model_area)]
    gauges.set_index('Gauge',inplace=True)


    periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
    if model_area_strict_match == True:
        periods = periods[periods.Model==model_area]
    else:
        periods = periods[periods.Model.str.contains(model_area)]

    report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
    report_text = report_text[report_text.Model.notna()]
    if model_area_strict_match == True:
        report_text = report_text[report_text.Model==model_area]
    else:
        report_text = report_text[report_text.Model.str.contains(model_area)]
    report_text.fillna('This section was left blank.', inplace=True)

    wwf_specs = pd.read_csv(wwf_csv)
    wwf_specs.set_index('Location',inplace=True)
    wwf_stats_specs = pd.read_csv('WWF_Stats_Specs.csv')

    diurnals = pd.read_csv('Diurnals.csv')

    if generate_confidence_csvs:

        map_periods = pd.read_excel(calibration_sheet,sheet_name="Periods_Map")
        map_periods = map_periods.loc[:, :'Model']
        if model_area_strict_match == True:
            map_periods = map_periods[map_periods.Model==model_area]
        else:
            map_periods = map_periods[map_periods.Model.str.contains(model_area)]

        thresholds = pd.read_excel(calibration_sheet,sheet_name="Thresholds",skiprows=1)
        thresholds.rename(columns={'Unnamed: 0':'Color'},inplace=True)
        for column in thresholds.columns[1:]:
            thresholds.rename(columns={column:column + ' ' + thresholds.loc[0,column]},inplace=True)
        for column in thresholds.columns[1:]:
            thresholds.rename(columns={column:column.replace('.1','')},inplace=True)
            thresholds.rename(columns={column:column.replace('.2','')},inplace=True)
        thresholds.drop(index=0,inplace=True)
        thresholds.drop(thresholds.index[3:],inplace=True) 
        thresholds.sort_values(by='DWF Flow Status',ascending=False,inplace=True)
        thresholds.reset_index(inplace=True)

        map_periods_check = map_periods.loc[:, :'Meter Status']
        map_periods_check_count = map_periods_check[map_periods_check['Meter Status']=='Primary']\
            [['Zone','Meter Status']].groupby(['Zone']).count()
        map_periods_check_count.rename(columns={'Meter Status':'Primary Count'},inplace=True)
        map_periods_check = pd.merge(map_periods_check,map_periods_check_count,on=['Zone'],how='left')
        multiple_primary = list(map_periods_check_count[map_periods_check_count['Primary Count']>1].index)

        missing_primary = []
        for zone in list(gauges.Location.unique()):
            if not zone in (map_periods_check_count.index):
                missing_primary.append(zone)

        if len(multiple_primary) > 0 or len(missing_primary) > 0 :
            error_message = 'Please correct the following errors in ' + os.path.basename(calibration_sheet) + ', sheet Periods_Map.\n\n'
            error_message += 'The following zones have multiple primary meters: '  + ','.join(multiple_primary) + '.\n'
            error_message += 'The following zones have no primary meters: '  + ','.join(missing_primary) + '.\n'
            raise ValueError(error_message)

except Exception as e: 
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 3\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")

In [ ]:
#PERMANENT CELL 4
#List all network elements to be imported.

try:
    nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
    nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
    level_nodes = nodes1 + nodes2
    #Convert all to string
    level_nodes = [str(x) for x in level_nodes]
    #Make set to clear duplicates
    level_nodes = set(level_nodes)

    ds_level_pipes = gauges[gauges['Level Type']=='Link(DS)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
    us_level_pipes = gauges[gauges['Level Type']=='Link(US)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
    flow_pipes = gauges.Pipe.dropna().unique().flatten().tolist()

    #Convert all to string
    ds_level_pipes = [str(x) for x in ds_level_pipes]
    us_level_pipes = [str(x) for x in us_level_pipes]
    flow_pipes = [str(x) for x in flow_pipes]

    #Add outfalls
    outfalls = pd.read_csv(outfall_csv,dtype={'Weir': str,'Outfall': str})
    outfalls['Res_ID'] = ''
    for index, row in outfalls.iterrows():
        prefix = ''
        if row['Layer'].lower() != 'msm_link' and row['Layer'].lower() != 'summation':
            prefix = row['Layer'][4:] + ':'
        muid = prefix + row['Weir']

        if row['Layer'].lower() != 'summation':
            outfalls.iloc[index,3] = row['Layer'][4:]
        outfalls.iloc[index,4] = muid

        if row['Layer'].lower() != 'summation':
            flow_pipes.append(muid)

    catchments = set()
    summation_df = pd.read_csv(summation_csv,dtype={'MUID': str,'SUMTO': str})        
    for index, row in summation_df.iterrows():
        prefix = ''

        if row['Layer'].lower() != 'msm_link' and row['Layer'].lower() != 'ms_catchment' and row['Layer'].lower() != 'summation':
            prefix = row['Layer'][row['Layer'].find('_') + 1:] + ':'

        muid = prefix + row['MUID']

        summation_df.iloc[index,0] = muid

        if '-Negative' in muid:
            muid = muid[:-9]

        if row['Layer'].lower() == 'ms_catchment':
            catchments.add(muid)
        else:
            flow_pipes.append(muid)

    #Make set to clear duplicates
    ds_level_pipes = set(ds_level_pipes)
    us_level_pipes = set(us_level_pipes)
    flow_pipes = set(flow_pipes)

except Exception as e: 
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 4\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")



In [ ]:
#PERMANENT CELL 5
#Import dfs0 files

try:

    #Import rain gauges
    rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()
    warnings = []

    #Import flow/level gauges
    gauge_ids = []
    first_dfs0 = True
    for dfs0_folder in dfs0_folders:
        for f in os.listdir(dfs0_folder):
            if f[-5:]=='.dfs0':


                if 'ps' in f.lower() and f[:2].lower() != 'ps':
                    gauge_id = re.split(r'[.]',f)[0]
                else:
                    gauge_id = re.split(r'_|[.]',f)[0]
                res = mikeio.read(dfs0_folder + '\\' + f)
                ts = res.to_dataframe()

                gauge_ids.append(gauge_id)

                first_level = True
                second_level = True
                first_velocity = True
                for i, column in enumerate(ts.columns):
                    if i == 0:
                        ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
                        if 'meter pow 3 per sec' in str(res.items[0]):
                            ts.Flow = ts.Flow * 1000
                        elif 'liter per sec' in str(res.items[0]):
                            ts.Flow = ts.Flow 
                        else:
                            warnings.append('First item in ' + f + ' does not appear to be type Discharge. This is not imported.')
                            ts.Flow = np.nan

                    elif str(res.items[i])[-21:] == '<Water Level> (meter)': 
                        if first_level == True:
                            ts.rename(columns={ts.columns[i]:'Level'},inplace=True)
                            first_level = False
                        elif second_level == True:
                            ts.rename(columns={ts.columns[i]:'Level2'},inplace=True)
                            second_level = False


                    elif str(res.items[i])[-31:] == '<Flow velocity> (meter per sec)' and first_velocity == True:
                        ts.rename(columns={ts.columns[i]:'Velocity'},inplace=True)
                        first_velocity = False

                if not 'Flow' in ts.columns:
                    ts['Flow'] = np.nan                     
                if not 'Velocity' in ts.columns:
                    ts['Velocity'] = np.nan
                if not 'Level' in ts.columns:
                    ts['Level'] = np.nan
                if not 'Level2' in ts.columns:
                    ts['Level2'] = np.nan

                ts['Gauge'] = gauge_id
                ts = ts[['Gauge','Flow','Level','Level2','Velocity']]
                ts['Seconds'] = ts.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill').astype('int64')
                ts['Volume'] = ts.Flow * ts.Seconds / 1000
                if first_dfs0 == True:
                    measured = ts.copy()
                else:
                    measured = pd.concat([measured,ts])
                first_dfs0 = False

    for warning in warnings:
        print (warning)

except Exception as e: 
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 5\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")

In [ ]:
#PERMANENT CELL 6
#Import network (and runoff if applicable) and addout result files

try:
    node_zones = pd.read_csv(node_csv,dtype={'Node': str})
    node_zones.set_index(node_zones.Node,inplace=True)
    spill_zones = list(node_zones.Zone.unique())

    first_round = True
    for f in os.listdir(result_folder):
        if '.ADDOUT.res1d' in f or ('.sqlite' in model and 'Network_HD.res1d' in f):

            res1d = Res1D(result_folder + '\\' + f)
            flood_types = ['WaterFlowRateAboveGround','WaterSpillDischarge']
            cover_types = ['Normal','Spilling']

            first_round_in_result = True
            res_zones = set()
            for node in res1d.data.Nodes:
                muid = node.Id
                for i, flood_type in enumerate(flood_types):
                    ts = res1d.query.GetNodeValues(muid,flood_type)
                    col_name = muid + '-' + cover_types[i]
                    if ts != None:
                        if max(ts) > 0 or first_round_in_result == True:
                            spill_df = pd.DataFrame(index=res1d.time_index)
                            spill_df['Node'] = muid
                            zone = node_zones.loc[muid,'Zone']
                            spill_df['Zone'] = zone
                            spill_df['Spill'] = ts
                            spill_df['Spill'] = spill_df['Spill'] * 1000

                            res_zones.add(zone)

                            if first_round == True:
                                spill_df_all = spill_df.copy()
                            else:
                                spill_df_all = pd.concat([spill_df_all,spill_df])                                                        
                            first_round = False
                            first_round_in_result = False

            #Create empty df for any zone not represented to show 0 on the graph.
            for spill_zone in spill_zones:
                if not spill_zone in res_zones:
                    spill_df = pd.DataFrame(index=res1d.time_index)
                    spill_df['Node'] = 'X'
                    spill_df['Zone'] = spill_zone
                    spill_df['Spill'] = 0           
                    spill_df_all = pd.concat([spill_df_all,spill_df])                                                        

    spill_df_zones = spill_df_all.copy()
    spill_df_zones['Date_Time'] = spill_df_zones.index
    spill_df_zones.drop(columns=['Node'],inplace=True)
    spill_df_zones = spill_df_zones.groupby(['Zone','Date_Time']).sum()
    spill_df_zones.reset_index(inplace=True)

    first_runoff = True
    if len(catchments) > 0:
        for f in os.listdir(result_folder):
            if 'RR.res1d' in f:

    #             print('Opening ' + f)
                res1d = Res1D(result_folder + '\\' + f)

                for catchment in catchments:
    #                 print('Importing ' + catchment)
                    catchment_df = pd.DataFrame(index = res1d.time_index)
                    catchment_df['ResultFile'] = f
                    catchment_df['MUID'] = catchment
                    catchment_df['DateTimeRef'] = catchment_df.index
                    catchment_df['Discharge'] = res1d.query.GetCatchmentValues(catchment, "TotalRunOff")  
                    catchment_df['Discharge'] = catchment_df['Discharge']*1000

                    catchment_df['Seconds'] = catchment_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill').astype('int64')
                    catchment_df['Volume'] = catchment_df.Discharge * catchment_df.Seconds / 1000
                    catchment_df.drop(columns=['Seconds'],inplace=True)

                    if first_runoff == True:
                        catchment_df_all = catchment_df.copy()
                    else:
                        catchment_df_all = pd.concat([catchment_df_all,catchment_df])
                    first_runoff = False

        result_specs = pd.read_csv(result_specs_csv)
        result_specs.rename(columns={'Runoff':'ResultFile'},inplace=True)
        catchment_df_all = pd.merge(catchment_df_all,result_specs,how='left',on=['ResultFile'])
        catchment_df_all.drop(columns='ResultFile',inplace=True)
        catchment_df_all.rename(columns={'Network':'ResultFile'},inplace=True)
        catchment_df_all = catchment_df_all[['ResultFile', 'MUID', 'DateTimeRef', 'Discharge', 'Volume']]

    results = []
    first_level = True
    first_flow = True
    first_velocity = True

    for f in os.listdir(result_folder):
        if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f and not 'hotstart' in f.lower():
            res1d = Res1D(result_folder + '\\' + f)
            reaches = res1d.data.Reaches
            nodes = res1d.data.Nodes

    #         print ("Importing network " + f + " at " + str(dt.datetime.now()))

            for i, node in enumerate(nodes):

                muid = node.Id
                if muid in level_nodes:
    #                 print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))

                    level_df = pd.DataFrame(index = res1d.time_index)
                    level_df['ResultFile'] = f
                    level_df['MUID'] = muid
                    level_df['Level'] = res1d.query.GetNodeValues(muid, "WaterLevel")                          

                    level_df['DateTimeRef'] = level_df.index

                    if first_level == True:
                        level_df_all = level_df.copy()
                    else:
                        level_df_all = pd.concat([level_df_all,level_df])
                    first_level = False


            first_round = True
            for i, reach in enumerate(reaches):

                muid = reach.Id[:reach.Id.rfind('-')]

                if muid in us_level_pipes or muid in ds_level_pipes:

                    if muid in us_level_pipes: 
                        values = res1d.query.GetReachStartValues(muid, "WaterLevel")
                    else:
                        values = res1d.query.GetReachEndValues(muid, "WaterLevel")

                    level_df = pd.DataFrame(index = res1d.time_index)
                    level_df['ResultFile'] = f
                    level_df['MUID'] = muid
                    level_df['Level'] = values                          

                    level_df['DateTimeRef'] = level_df.index

                    if first_level == True:
                        level_df_all = level_df.copy()

                    else:
                        level_df_all = pd.concat([level_df_all,level_df])
                    first_level = False



            for i, reach in enumerate(reaches):

                muid = reach.Id[:reach.Id.rfind('-')]

                if muid in flow_pipes or muid in list(summation_df.MUID):

                    values = res1d.query.GetReachEndValues(muid, "Discharge")
                    flow_df = pd.DataFrame(index = res1d.time_index)
                    flow_df['ResultFile'] = f
                    flow_df['MUID'] = muid
                    flow_df['DateTimeRef'] = flow_df.index
                    flow_df['Discharge'] = values                          
                    flow_df['Discharge'] = flow_df['Discharge'] * 1000

                    flow_df['Seconds'] = flow_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill').astype('int64')
                    flow_df['Volume'] = flow_df.Discharge * flow_df.Seconds / 1000
                    flow_df.drop(columns=['Seconds'],inplace=True)

                    if first_flow == True:
                        flow_df_all = flow_df.copy()

                    else:
                        flow_df_all = pd.concat([flow_df_all,flow_df])
                    first_flow = False

            for i, reach in enumerate(reaches):

                muid = reach.Id[:(-1 * len(str(i)) - 1)]

                if muid in flow_pipes:
                    print ("Importing pipe " + reach.Id + " velocity at " + str(dt.datetime.now()))

                    values = res1d.query.GetReachEndValues(muid, "FlowVelocity")
                    velocity_df = pd.DataFrame(index = res1d.time_index)
                    velocity_df['ResultFile'] = f
                    velocity_df['MUID'] = muid
                    velocity_df['DateTimeRef'] = velocity_df.index
                    velocity_df['Velocity'] = values                          


                    if first_velocity == True:
                        velocity_df_all = velocity_df.copy()
                    else:
                        velocity_df_all = pd.concat([velocity_df_all,velocity_df])
                    first_velocity = False

            first_round = False

    if len(catchments) > 0:
        catchment_df_all_filter  = catchment_df_all.copy()
        times = list(flow_df_all.DateTimeRef.unique())
        catchment_df_all_filter.set_index(catchment_df_all_filter.DateTimeRef,inplace=True)
        catchment_df_all_filter =  catchment_df_all_filter[catchment_df_all_filter['DateTimeRef'].isin(times)]
        flow_df_all = pd.concat([flow_df_all,catchment_df_all_filter])

    negatives = list(summation_df.query("MUID.str.endswith('-Negative')").MUID.unique())
    original_negatives = []
    for negative in negatives:
        original_negatives.append(negative[:-9])
    negatives_df = flow_df_all[flow_df_all.MUID.isin(original_negatives)].copy()
    negatives_df.MUID = negatives_df.MUID + '-Negative'
    negatives_df.Discharge = negatives_df.Discharge * -1
    negatives_df.Volume = negatives_df.Volume * -1

    flow_df_all = pd.concat([flow_df_all,negatives_df])

    #Summation
    df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
    df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum','Volume':'sum'})
    df_result_sum.reset_index(inplace=True)
    df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
    df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)

    flow_df_all = pd.concat([flow_df_all,df_result_sum])

    not_founds = []
    for muid in summation_df.MUID.unique():
        if not muid in flow_df_all.MUID.unique():
            not_founds.append(muid)

    if len(not_founds) > 0:
        error_message = 'The following elements in summation_df were not found: \n' 
        error_message += ','.join(not_founds) + '.\n'
        raise ValueError(error_message)

except Exception as e: 
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 6\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")

In [ ]:
#PERMANENT CELL 7
#Create HTML reports

try:
    
    shutil.copy2('style.css', output_folder + '\\style.css')
    shutil.copy2('script.js', output_folder + '\\script.js')
    
    error_list = []
    vol_stat_list = []
    
    for gauge in gauges.index:
        zone = str(gauges.loc[gauge,'Location'])
        if gauge in list(periods.Meter) and (zone in zone_filter or len(zone_filter) == 0):
            location_type = gauges.loc[gauge,'Location Type']

            pipe = str(gauges.loc[gauge,'Pipe'])
            node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
            rain_gauge = str(gauges.loc[gauge,'Rain Gauge'])
            MonitoringType = str(gauges.loc[gauge,"Sensor Output"])
            NodeID = str(gauges.loc[gauge, "Node1 (Or Pipe if pipe level)"])
            MHName = str(gauges.loc[gauge,'Node1 AssetName']) 
            number_catchments = wwf_specs.loc[zone,'Number sanitary'] \
                + wwf_specs.loc[zone,'Number combined'] \
                + wwf_specs.loc[zone,'Number stormwater']

            effluent_types = []
            if wwf_specs.loc[zone,'Number sanitary'] > 0:
                effluent_types.append('Sanitary')
            if wwf_specs.loc[zone,'Number combined'] > 0:
                effluent_types.append('Combined')
            if wwf_specs.loc[zone,'Number stormwater'] > 0:
                effluent_types.append('Storm')

            for i, effluent_type in enumerate(effluent_types):
                if i == 0:
                    effluent_type_string = effluent_type
                elif i == len(effluent_types)-1:
                    effluent_type_string += ' and ' + effluent_type
                else:
                    effluent_type_string += ', ' + effluent_type          


            local_population = 0
            for ww_type in ww_types:
                if ww_type in res_types:                
                    local_population += dwf_specs.loc[zone,ww_type + '_Population']

            total_area = round(wwf_specs.loc[zone,'Drainage area (ha)'],1)

            PipeShape = str(gauges.loc[gauge,'Pipe Shape'])
            try:
                PipeDimension = str(round(gauges.loc[gauge, 'Pipe Dimension'],2))
            except:
                PipeDimension = gauges.loc[gauge, 'Pipe Dimension']
                
            tabs = ['Report']
            for p_no, period_spec in enumerate(period_specs):
                description = period_spec[:2] + ' Calibration'
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                if not pd.isnull(start):
                    if start.month == end.month:
                        tabs.append(period_spec[:3] + ', ' + start.strftime("%b %d") + ' - ' + end.strftime("%d %Y"))
                    else:
                        if start.year == end.year:
                            tabs.append(period_spec[:3] + ', ' + start.strftime("%b %d") + ' - ' + end.strftime("%b %d %Y"))                
                        else:
                            tabs.append(period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y"))                


            with open(output_folder + "\\" + "Calibration_Report_" + gauge + ".html", 'w') as f:                
                
                f.write('<!DOCTYPE html>\n')
                f.write('<html>\n')
                f.write('<head>\n')
                f.write('<meta charset="utf-8">\n')
                f.write('<script src="script.js"></script>\n')
                f.write('<link rel="stylesheet" href="style.css">\n')
                f.write('</head>\n')
                f.write('<body>\n\n')
                
                f.write('<div class="tab">\n')
                for tab in tabs:
                    f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
                f.write('</div>\n')
                
                f.write('<div id="' + tabs[0] + '" class="tabcontent">\n') 
                
                f.write('<h1 style="text-align:center">Model Calibration Report for gauge ' 
                        + gauge + ' in Zone ' + zone + ', ' + model_area + '</h1>')
            
                f.write('<h2>1. Description of the Calibration Area</h2>')
                f.write('<table class="first">\n')
                f.write('<tr>\n')
                f.write('<td>Number of Catchments</td>\n')
                f.write('<td>'+ str(number_catchments) +'</td>\n')
                f.write('</tr>\n')
                f.write('<tr>\n')
                f.write('<td>Zone Area (Ha)</td>\n')
                f.write('<td>'+ str(total_area) +'</td>\n')
                f.write('</tr>\n')
                f.write('<tr>\n')
                f.write('<td>Local Residential Population</td>\n')
                f.write('<td>'+ str(int(local_population)) +'</td>\n')
                f.write('</tr>\n')
                f.write('<tr>\n')
                f.write('<td>Effluent Type</td>\n')
                f.write('<td>'+ effluent_type_string +'</td>\n')
                f.write('</tr>\n')            
                for ww_type in ww_types:
                    if ww_type not in res_types:
                        f.write('<tr>\n')
                        f.write('<td>' + ww_type + ' Area (Ha)</td>\n')
    #                     f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_' + description],1)) +'</td>\n')
                        f.write('<td>'+ str(round(dwf_specs.loc[zone,ww_type + '_Area'],1)) +'</td>\n')
                        f.write('</tr>\n')
                f.write('</tr>\n')
                f.write('</table>')

                f.write('<h2>2. Description of the Sensor and Network</h2>')            
                f.write('<table class="first">\n')
                f.write('<tr>\n')
                f.write('<td>'+ 'Monitoring Type' +'</td>\n')
                f.write('<td>'+ MonitoringType +'</td>\n')
                f.write('</tr>\n')
                f.write('<tr>\n')
                f.write('<td>'+ 'Node ID' +'</td>\n')
                f.write('<td>'+ node +'</td>\n')
                f.write('</tr>\n')
                f.write('<tr>\n')
                f.write('<td>'+ 'Pipe ID' +'</td>\n')
                f.write('<td>'+ pipe +'</td>\n')
                f.write('</tr>\n')
                f.write('<tr>\n')
                f.write('<td>'+ 'Pipe Shape' +'</td>\n')
                f.write('<td>'+ PipeShape +'</td>\n')
                f.write('</tr>\n')
                f.write('</tr>\n')
                f.write('<tr>\n')
                f.write('<td>'+ 'Pipe Dimension (m) or CRS ID ' +'</td>\n')
                f.write('<td>'+ PipeDimension +'</td>\n')
                f.write('</tr>\n')
                f.write('</table>')


                f.write('<h2>3. Model Calibration Setup: </h2>')

                f.write('<table class="first">\n')
                f.write('  <tr>\n')
                f.write('    <th>Simulation</th>\n')
                f.write('    <th>Start Date</th>\n')
                f.write('    <th>End Date</th>\n')
                f.write('    <th>Duration</th>\n')
                f.write('  </tr>\n')

                for period_spec in period_specs:
                    description = period_spec[:2] + ' Calibration'
                    start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                    end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                    if pd.isnull(start):
                        break
                    duration = str((end - start).days) + ' days'

                    f.write('  <tr>\n')
                    f.write('    <td>' + description + '</th>\n')
                    f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                    f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>\n')
                    f.write('    <td>' + duration + '</th>\n')
                    f.write('  </tr>\n')                   
                f.write('</table>\n')

                f.write('<h2>4. Zone Map</h2>\n')
                f.write('<img src="' + map_folder + '\\' + zone + '.jpg" alt="' + gauge + '">\n')

                f.write('<h2>6. Diurnal Patterns</h2>\n')  
                f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Diurnal Text'])[0]) + '</p>\n')
                fig_diurnal = go.Figure()
                diurnals_filter1 = diurnals[diurnals.Zone==zone]
                profile_ids = list(diurnals_filter1.Profile.unique())
                for profile_id in profile_ids:
                    diurnals_filter2 = diurnals_filter1[diurnals_filter1.Profile==profile_id]
                    fig_diurnal.add_trace(go.Scatter(x=diurnals_filter2.Sqn, 
                                                 y = diurnals_filter2.Multiplier, 
                                                 mode='lines',name=profile_id + ' (' + diurnals_filter2.iloc[0,2] + ')'))             
                fig_diurnal.update_layout(
                    autosize=False,
                    width = 1362,
                    height=250,
                    margin=dict(
                        l=50,
                        r=50,
                        b=25,
                        t=25,
                        pad=4
                        ),
    #                 yaxis_title="Discharge (L/s)" 
                    )

                f.write(fig_diurnal.to_html(full_html=False, include_plotlyjs='cdn'))

                f.write('<h2>6. DWF Calibration Summary</h2>\n')  
                f.write('<p>' + str(list(report_text[report_text.Zone==zone]['DWF Text'])[0]) + '</p>\n')
                f.write('<h2>7. WWF Calibration and Validation Summary</h2>\n')
                f.write('<p>' + str(list(report_text[report_text.Zone==zone]['WWF Text'])[0]) + '</p>\n')
                f.write('<h2>8. Calibration Issues</h2>\n')
                f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Issue Text'])[0]) + '</p>\n')
                f.write('<h2>9. Recommendations</h2>\n')
                f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Recommendation Text'])[0]) + '</p>\n')                
                f.write('</div>\n')

                wwf_plot_header_added = False
                for p_no, period_spec in enumerate(period_specs):
                    
                    

                    start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                    end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]

                    if pd.isnull(start):
                        break  
                        
                    f.write('<div id="' + tabs[p_no + 1] + '" class="tabcontent">\n') 

                    measured_gauge = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                                        
                    if location_type == 'PS':
                        measured_gauge['Discharge_Hourly'] = measured_gauge['Flow'].rolling('1h').mean()

                    measured_rain = rainfall.loc[(rainfall.index >= start) & (rainfall.index <= end)][rain_gauge].to_frame()

                    result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                    if len(result_flow_gauge.ResultFile.unique()) > 1:
                        result_file_counts = result_flow_gauge['ResultFile'].value_counts()
                        most_common_result_file = result_file_counts.idxmax()
                        result_flow_gauge = result_flow_gauge[result_flow_gauge['ResultFile'] == most_common_result_file]

                    result_velocity_gauge = velocity_df_all.loc[(velocity_df_all.MUID==pipe) & (velocity_df_all.index >= start) & (velocity_df_all.index <= end)].copy()
                    if len(result_velocity_gauge.ResultFile.unique()) > 1:
                        result_file_counts = result_velocity_gauge['ResultFile'].value_counts()
                        most_common_result_file = result_file_counts.idxmax()
                        result_velocity_gauge = result_velocity_gauge[result_velocity_gauge['ResultFile'] == most_common_result_file]
                    
                    spill_df_zone = spill_df_zones.loc[(spill_df_zones.Zone==zone) & (spill_df_zones.Date_Time >= start) & (spill_df_zones.Date_Time <= end)].copy()                
                    spill_df_all_zone = spill_df_all.loc[(spill_df_all.Zone==zone) & (spill_df_all.index >= start) & (spill_df_all.index <= end)].copy()
                    spill_df_all_zone.drop(columns=['Zone'],inplace=True)
                    spill_top_ten = spill_df_all_zone.groupby(['Node']).max()
                    spill_top_ten.reset_index(inplace=True)
                    spill_top_ten.sort_values(by='Spill',ascending=False,inplace=True)
                    spill_top_ten.reset_index(drop=True,inplace=True)
                    spill_top_ten = spill_top_ten[spill_top_ten['Spill']>0]
                    spill_top_ten = spill_top_ten.head(10)                                

                    if location_type == 'PS':
                        result_flow_gauge['Discharge_Hourly'] = result_flow_gauge['Discharge'].rolling('1h').mean()

                    result_level_gauge = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)]
                    if len(result_level_gauge.ResultFile.unique()) > 1:
                        result_file_counts = result_level_gauge['ResultFile'].value_counts()
                        most_common_result_file = result_file_counts.idxmax()
                        result_level_gauge = result_level_gauge[result_level_gauge['ResultFile'] == most_common_result_file]                    
                   
                    has_outfalls = False
                    if zone in list(outfalls.Zone):
                        has_outfalls = True
                        outfalls_filter = outfalls[outfalls.Zone==zone].copy()
                        outfalls_filter.reset_index(inplace=True)

                        for index, row in outfalls_filter.iterrows():
                            overflow_pipe = row['Res_ID']
                            result_outfalls = flow_df_all.loc[(flow_df_all.MUID==overflow_pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                            result_outfalls['Outfall'] = row['Outfall']
                            if index == 0:
                                result_outfalls_all = result_outfalls.copy()
                            else:
                                result_outfalls_all = pd.concat([result_outfalls_all,result_outfalls])

                #     result_gauge = df_result.loc[(df_result.index >= start) & (df_result.index <= end)]
                    #---------------------------------------------------------------------------------------------
                    compare_stats = []
                    peak_level_model = round(result_level_gauge.Level.max(),2)
                    compare_stats.append(['Peak HGL Model',peak_level_model,'m'])
                    peak_level_measured = round(measured_gauge.Level.max(),2)
                    compare_stats.append(['Peak HGL Measured',peak_level_measured,'m'])
                    compare_stats.append(['Peak HGL Difference',round(peak_level_model - peak_level_measured,2),'m'])
    #                 volume_model = round(result_flow_gauge.Volume.sum(),0).item()
                    volume_model = round(result_flow_gauge.Volume.sum(),0)
                    compare_stats.append(['Volume Model',volume_model,'m3'])
                    volume_measured = round(measured_gauge.Volume.sum(),0).item() #convert to native float to avoid inf
                    compare_stats.append(['Volume Measured',volume_measured,'m3'])
                    try:
                        vol_dif_pc = (volume_model - volume_measured) / volume_measured * 100
                        compare_stats.append(['Volume Difference',round(vol_dif_pc, 0),'%'])
                        if period_spec[:3] != "DWF":
                            vol_stat_list.append([gauge,vol_dif_pc,start,end])
                    except:
                        compare_stats.append(['Volume Difference','','%'])
                    peak_flow_model = round(result_flow_gauge.Discharge.max(),1)
                    compare_stats.append(['Peak Flow Model',peak_flow_model,'L/s'])
                    peak_flow_measured = round(measured_gauge.Flow.max(),1)
                    compare_stats.append(['Peak Flow Measured',peak_flow_measured,'L/s'])
                    try:
                        compare_stats.append(['Peak Flow Difference',round((peak_flow_model - peak_flow_measured) / peak_flow_measured * 100, 0),'%'])
                    except:
                        compare_stats.append(['Peak Flow Difference','','%'])


                    f.write('<h3> Gauge ' 
                        + gauge + ' in Zone ' + zone + '</h3>\n')  
                    
                    if period_spec[:3] == "DWF":

                        f.write('<div class="row">')

                        f.write('<div class="column">')    
                        f.write('<table class="second">\n')
                        f.write('  <tr>\n')
                        f.write('    <th>Description</th>\n')
                        f.write('    <th>Value</th>\n')
                        f.write('    <th>Unit</th>\n')
                        f.write('  </tr>\n')                   

                        #sim and measured stats
                        for compare_stat in compare_stats:
                            f.write('  <tr>\n')
                            f.write('    <td>' + compare_stat[0] + '</th>\n')
                            f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                            f.write('    <td>' + compare_stat[2] + '</th>\n')
                            f.write('  </tr>\n')

                        f.write('</table>\n')
                        f.write('</div>')                    

                        #Loading rate
                        f.write('<div class="column">')    
                        f.write('<table class="second">\n')
                        f.write('  <tr>\n')
                        f.write('    <th>Load Type</th>\n')
                        f.write('    <th>Loading Rate</th>\n')
                        f.write('    <th>Unit</th>\n')
                        f.write('  </tr>\n')

                        for ww_type in ww_types:
                            if ww_type in res_types:
                                decimals = 3
                            else: 
                                decimals = 1
                            rate = round(dwf_specs.loc[zone,ww_type + '_Rate'],decimals)
                            unit = all_types.loc[ww_type,'Unit2']

                            f.write('  <tr>\n')
                            f.write('    <td>' + ww_type + '</th>\n')
                            f.write('    <td>' + str(rate) + '</th>\n')
                            f.write('    <td>' + unit + '</th>\n')
                            f.write('  </tr>\n') 
                        f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('</table>\n')
                        f.write('</div>')

                        f.write('<div class="column">')

                        #Population and ICI areas
                        f.write('<table class="second">\n')
                        f.write('  <tr>\n')
                        f.write('    <th>Load Type</th>\n')
                        f.write('    <th>Local</th>\n')
                        if use_accumulation == True:
                            f.write('    <th>Total</th>\n')
                        else:
                            f.write('    <th></th>\n')
                        f.write('    <th>Unit</th>\n')
                        f.write('  </tr>\n')

                        for ww_type in ww_types:
                            if ww_type in res_types:
                                decimals = 0
                            else: 
                                decimals = 1
                            description = unit = all_types.loc[ww_type,'Description']
                            local_val = round(dwf_specs.loc[zone,ww_type + '_' + description],decimals)
                            if use_accumulation == True:
                                total_val = round(dwf_specs.loc[zone,ww_type + '_' + description + '_Upstream'],decimals)
                            unit = all_types.loc[ww_type,'Unit1']

                            f.write('  <tr>\n')
                            f.write('    <td>' + ww_type + '</th>\n')
                            f.write('    <td>' + str(local_val) + '</th>\n')
                            if use_accumulation == True:
                                f.write('    <td>' + str(total_val) + '</th>\n')
                            else:
                                f.write('    <td></th>\n')
                            f.write('    <td>' + unit + '</th>\n')
                            f.write('  </tr>\n') 
                        f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('<tr><td><br/></td><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                        f.write('</table>\n')
                        f.write('</div>')

                        f.write('<div class="column">')
                        #Waterload
                        f.write('<table class="second">\n')
                        f.write('  <tr>\n')
                        f.write('    <th>Load Type</th>\n')
                        f.write('    <th>Local</th>\n')
                        if use_accumulation == True:
                            f.write('    <th>Total</th>\n')
                        else:
                            f.write('    <th></th>\n')
                        f.write('    <th>Unit</th>\n')
                        f.write('  </tr>\n')

                        for load_type in all_types.index.values.tolist():

                            local_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad'],0)
                            if use_accumulation == True:
                                total_val = round(dwf_specs.loc[zone,load_type + '_WaterLoad_Upstream'],0)
                            unit = 'm3/d'

                            f.write('  <tr>\n')
                            f.write('    <td>' + load_type + '</th>\n')
                            f.write('    <td>' + str(local_val) + '</th>\n')
                            if use_accumulation == True:
                                f.write('    <td>' + str(total_val) + '</th>\n')
                            else:
                                f.write('    <td></th>\n')
                            f.write('    <td>' + unit + '</th>\n')
                            f.write('  </tr>\n') 

                        f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')

                        f.write('</table>\n')
                        f.write('</div>')
    #                     f.write('</div>')

                    else:

                        total_rows = 10
                        f.write('<div class="row">')
                        for i in range(0,4): #Cycle through the three table types.   
                            f.write('<div class="column">')    
                            f.write('<table class="second">\n')
                            f.write('  <tr>\n')
                            f.write('    <th>Description</th>\n')
                            f.write('    <th>Value</th>\n')
                            f.write('    <th>Unit</th>\n')
                            f.write('  </tr>\n')

                            current_row = 0
                            if i == 0:
                                for compare_stat in compare_stats:
                                    f.write('  <tr>\n')
                                    f.write('    <td>' + compare_stat[0] + '</th>\n')
                                    f.write('    <td>' + str(compare_stat[1]) + '</th>\n')
                                    f.write('    <td>' + compare_stat[2] + '</th>\n')
                                    f.write('  </tr>\n')
                                    current_row += 1                
                            else:
                                for index, row in wwf_stats_specs[wwf_stats_specs.Table==i].iterrows():

                                    decimals = row['Decimals']


                                    f.write('  <tr>\n')
                                    f.write('    <td>' + row['Description'] + '</th>\n')
                                    if row['DF'] == 'wwf_specs':
                                        if slope_source_unit_meter_per_meter == False and row['Description'] == 'Average slope':
                                            f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']]/1000,decimals)) + '</th>\n')
                                        elif 'max (mm)' in row['DF_Col']:
                                            f.write('    <td>' + str(int(wwf_specs.loc[zone,row['DF_Col']]*1000)) + '</th>\n')
                                        elif decimals > 0:
                                            f.write('    <td>' + str(round(wwf_specs.loc[zone,row['DF_Col']],decimals)) + '</th>\n')
                                        else:
                                            f.write('    <td>' + str(int(wwf_specs.loc[zone,row['DF_Col']])) + '</th>\n')
                                    else: 
                                        f.write('    <td>' + str(round(measured_rain[rain_gauge].sum(),decimals)) + '</th>\n')
                                    f.write('    <td>' + row['Unit'] + '</th>\n')
                                    f.write('  </tr>\n')
                                    current_row += 1

                            remaining_rows = total_rows - current_row

                            for remaining_row in range(remaining_rows):
                                f.write('<tr><td><br/></td><td><br/></td><td><br/></td></tr>\n')
                            f.write('</table>\n')
                            f.write('</div>')

                    #Spill top 10
                    if period_spec[:3] == "DWF":
                        total_rows = 9
                    else:
                        total_rows = 10

                    f.write('<div class="row">')

                    f.write('<div class="column">')    
                    f.write('<table class="second">\n')
                    f.write('  <tr>\n')
                    f.write('    <th>Node Spill Top 10</th>\n')
                    f.write('    <th>Peak Spill (L/s)</th>\n')
                    f.write('  </tr>\n')

                    current_row = 0

                    for index, row in spill_top_ten.iterrows():
                        f.write('  <tr>\n')
                        f.write('    <td>' + row['Node'] + '</th>\n')
                        f.write('    <td>' + str(int(row['Spill'])) + '</th>\n')
                        f.write('  </tr>\n')
                        current_row += 1                


                    remaining_rows = total_rows - current_row

                    for remaining_row in range(remaining_rows):
                        f.write('<tr><td><br/></td><td><br/></td></tr>\n')
                    f.write('</table>\n')
                    f.write('</div>')


                    f.write('</div>')
                        #----------------------------------------------------------------------------------------------


                    fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                        'Discharge at Gauge ' + gauge + ', Reach ' + pipe + ', Rain Gauge ' + rain_gauge,  
                                                        'HGL at Gauge ' + gauge + ', Node ' + node),
                                                        specs=[[{"secondary_y": True}, {"secondary_y": True}]],
                                                       horizontal_spacing = 0.1)

                    fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge, mode='lines',name='Model Flow'),1,1)   
                    fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Flow, mode='lines',name='Measured Flow'),1,1)
                    fig.add_trace(go.Scatter(x=measured_rain.index, y=measured_rain[rain_gauge], mode='lines',name='Rainfall'),1,1, secondary_y=True)

                    if location_type == 'PS':
                        fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge_Hourly, mode='lines',name='Model Flow Hourly',line_color='yellow'),1,1)   
                        fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Discharge_Hourly, mode='lines',name='Measured Flow Hourly',line_color='black'),1,1)

                    fig.add_trace(go.Scatter(x=result_level_gauge.index, y=result_level_gauge.Level, mode='lines',name='Model Level',line_color='green'),1,2)
                    fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Level, mode='lines',name='Measured Level',line_color='purple'),1,2)
                    
                    invert = network_specs.loc[node,'Invert']
                    ground = network_specs.loc[node,'Ground']
                    soh = network_specs.loc[node,'SOH']
                    
                    fig.add_trace(go.Scatter(x=[start,end], y=[invert,invert], mode='lines',name='Invert Level',line={'color': 'black','dash': 'dash'}),1,2)
                    fig.add_trace(go.Scatter(x=[start,end], y=[ground,ground], mode='lines',name='Ground Level',line={'color': 'brown','dash': 'dash'}),1,2)
                    if not math.isnan(soh):
                        fig.add_trace(go.Scatter(x=[start,end], y=[soh,soh], mode='lines',name='SOH',line={'color': 'red','dash': 'dash'}),1,2)         
                    
                    chart_header = period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y")
                    
                    fig.update_layout(
                        autosize=False,
                        width = 1600,
                        height=280,
                        margin=dict(
                            l=0,
                            r=0,
                            b=25,
                            t=60,
                            pad=4
                            ),

                        title ={
                            'text' : chart_header,
                            'x':0.45,
                            'xanchor': 'center'
                        })

                    fig['layout']['yaxis']['title']='Discharge (L/s)'
                    fig['layout']['yaxis2']['title']='Rainfall (mm / 5 min)' 
                    fig['layout']['yaxis2']['range']=[10,0]
                    fig['layout']['yaxis3']['title']='HGL (m)'

                    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                    f.write('<br>')

                    fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=(
                                                        'Velocity at Gauge ' + gauge + ', Reach ' + pipe,
                                                        'Total Manhole Spills, Weir Overtopping, CSO and SSO'),
                                                        specs=[[{"secondary_y": False}, {"secondary_y": False}]],
                                                       horizontal_spacing = 0.1)

                    fig.add_trace(go.Scatter(x=result_velocity_gauge.DateTimeRef, y=result_velocity_gauge.Velocity, mode='lines',name='Model Velocity'),1,1)   
                    fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Velocity, mode='lines',name='Measured Velocity'),1,1)

                    spill_df_zone
                    fig.add_trace(go.Scatter(x=spill_df_zone.Date_Time, y=spill_df_zone.Spill, mode='lines',name='Total Manhole Spilling'),1,2)
                    if has_outfalls == True:

                        show_first = []
                        show_last = []
                        for outfall in list(outfalls_filter.Outfall):
                            if outfall in measured.Gauge.unique():
                                show_first.append(outfall)
                            else:
                                show_last.append(outfall)
                        outfalls_to_plot = show_first + show_last
                        for i, outfall in enumerate(outfalls_to_plot):
                            result_outfalls_all_filter = result_outfalls_all[result_outfalls_all.Outfall==outfall]
                            if i < len(colors):
                                fig.add_trace(go.Scatter(x=result_outfalls_all_filter.index, 
                                                         y = result_outfalls_all_filter.Discharge, 
                                                         mode='lines',name=outfall,line={'color': colors[i]}),1,2)
                            else:
                                fig.add_trace(go.Scatter(x=result_outfalls_all_filter.index, 
                                                         y = result_outfalls_all_filter.Discharge, 
                                                         mode='lines',name=outfall),1,2)

                            if outfall in measured.Gauge.unique():
                                outfall_measured = measured.loc[(measured.Gauge==outfall) & (measured.index >= start) & (measured.index <= end)].copy()    
                                if i < len(colors):
                                    fig.add_trace(go.Scatter(x=outfall_measured.index, 
                                                             y=outfall_measured.Flow, 
                                                             mode='lines',name=outfall + ' - Measured',
                                                             line={'color': colors[i],'dash': 'dash'}),1,2)                                                         
                                else:
                                    fig.add_trace(go.Scatter(x=outfall_measured.index, 
                                                             y=outfall_measured.Flow, 
                                                             mode='lines',name=outfall + ' - Measured',
                                                             line={'dash': 'dash'}),1,2)


                    fig.update_layout(
                        autosize=False,
                        width = 1600,
                        height=280,
                        margin=dict(
                            l=0,
                            r=0,
                            b=25,
                            t=60,
                            pad=4
                            ))


                    fig['layout']['yaxis']['title']='Velocity (m/s)'
                    fig['layout']['yaxis2']['title']='Discharge (L/s)'

                    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
                    f.write('<br>\n')
                    
                    f.write('</div>\n')
                    f.write('</div>\n')
                    
                f.write('</body>\n')
                f.write('</html>\n')

            f.close()
            print("Writing html for " + gauge)
            
    #Create csv file for suggested WWF dates for confidence maps based on smallest volume differences.
    if generate_confidence_csvs:
        vol_stat_list_df = pd.DataFrame(vol_stat_list,columns=['Gauge','Diff','Start','End'])
        vol_stat_list_df = vol_stat_list_df[~np.isinf(vol_stat_list_df['Diff'])]
        vol_stat_list_df.Diff = abs(vol_stat_list_df.Diff)
        vol_stat_list_df.sort_values(['Gauge','Diff'],inplace=True)
        vol_stat_list_df = vol_stat_list_df.groupby('Gauge').head(2).reset_index(drop=True)
        vol_stat_list_df.reset_index(inplace=True,drop=True)
        vol_stat_list_df = (vol_stat_list_df
                      .set_index(['Gauge', vol_stat_list_df.groupby('Gauge').cumcount() + 1])
                      .unstack()
                      .sort_index(axis=1, level=1))  # Ensure columns are in the correct order
        vol_stat_list_df.columns = [f"{col}{num}" for col, num in vol_stat_list_df.columns]
        vol_stat_list_df = vol_stat_list_df.reset_index()
        vol_stat_list_df = vol_stat_list_df[['Gauge', 'Diff1','Diff2','Start1','End1','Start2','End2']]
        vol_stat_list_df.to_csv(output_folder + '\\Suggested_Confidence_Dates_Smallest_Volume_Differences.csv',index=False)

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 7', b'Error', 0)
    raise ValueError("Error")




In [ ]:
#PERMANENT CELL 8
#Create csv files for confidence maps.

try:
    
    if generate_confidence_csvs == True:
        
        # sql function
        def sql_to_df(sql,model):
            con = sqlite3.connect(model)
            df = pd.read_sql(sql, con)
            con.close()
            return df

        sql = "SELECT muid, invertlevel FROM msm_Node WHERE active=1"
        node_df = sql_to_df(sql, model)
        node_df.set_index(['muid'], inplace=True)

        sql = "SELECT muid, uplevel, dwlevel, fromnodeid, tonodeid, " 
        sql += "CASE typeno WHEN 1 THEN diameter WHEN 2 THEN height ELSE 0 END AS pipe_height "
        sql += "FROM msm_Link WHERE active=1 "
        pipe_df = sql_to_df(sql, model)
        pipe_df.set_index(['muid'], inplace=True)
    

        cols = ['Gauge', 'Zone', 'Pipe', 'Node', 'Pipe Shape', 'Node Bottom Level (m)', 'Pipe Bottom Level (m)', 
                'Pipe Top Level (m)', 'Diameter/Height (m)', 'Result File', 'Start', 'End', 'Used for', 'Meter Rank', 
                'Model Peak Hourly Average Flow (L/s)', 'Measured Peak Hourly Average Flow (L/s)', 'Peak Difference Flow (L/s)',
                'Peak Difference Flow (%)', 'Model Peak HGL (m)', 'Measured Peak HGL (m)', 'Peak Difference HGL (m)',
                'Model Pipe Filled', 'Measured Pipe Filled', 'Model Volume (m3)', 'Measured Volume (m3)',
                'Volume Difference (m3)', 'Volume Difference (%)', 'Threshold Category','HGL Status', 
                'Flow Status', 'Volume Status', 'Weighted Status', 'HGL Status Value', 'Flow Status Value', 
                'Volume Status Value', 'Flow Volume Average Status Value', 'Weighted Status Value', 'WL QA', 'Flow QA']
        dtypes = ['object', 'object', 'object', 'object', 'object', 'float64', 'float64', 
                  'float64', 'float64', 'object', 'datetime64[ns]', 'datetime64[ns]', 'object', 'object', 
                  'float64', 'float64', 'float64', 
                  'object', 'object', 'float64', 'float64', 
                  'float64', 'float64', 'object', 'float64', 'object',
                  'object', 'object', 'object', 'float64', 'float64', 
                  'float64', 'float64', 'float64', 'object', 'object']
        col_specs = list(zip(cols,dtypes))
        data_types = {col[0]: col[1] for col in col_specs}
        summary_df = pd.DataFrame(columns=cols).astype(data_types)
      
        color_dict = {}
        color_dict[1] = 'Green'
        color_dict[1.5] = 'Light Green'
        color_dict[2] = 'Yellow'
        color_dict[2.5] = 'Light Red'
        color_dict[3] = 'Red'

        event_categories = ['DWF Calibration','WWF Calibration','WWF Validation']

        result_types = ['HGL','Flow','Volume']

        result_points = []
        zone_review = []

        zone_dict = {}
        for index, row in map_periods.iterrows():
            gauge = row['Meter']
            node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
            pipe = str(gauges.loc[gauge,'Pipe'])
            zone = row['Zone']
            zone_dict[gauge] = [zone,row['Meter Status']]
            if zone in zone_filter or len(zone_filter) == 0:
                meter_status = row['Meter Status']

                x = gauges.loc[gauge,'X coordinate']
                y = gauges.loc[gauge,'Y coordinate']
                x_shift = gauges.loc[gauge,'Shift X (m)']
                y_shift = gauges.loc[gauge,'Shift Y (m)']
                catchment_type = gauges.loc[gauge,'Catchment Type']

                print('Processing ' + gauge + ' for confidence maps.')

                for i, event_category in enumerate(event_categories):
                    event_category_short = event_category[:7]
                    start = row[event_category_short + ' Start']
                    end = row[event_category_short + ' End']

                    #Find the associated result file
                    result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                    if len(result_flow_gauge.ResultFile.unique()) > 1:
                        result_file_counts = result_flow_gauge['ResultFile'].value_counts()
                        result_file = result_file_counts.idxmax()

                    else:
                        result_file = result_flow_gauge.ResultFile.unique()

                    row_index = gauge + ' ' + event_category
                    new_row_df = pd.DataFrame({'Gauge': gauge, 
                                               'Used for':event_category,
                                               'Zone': zone,
                                               'Start':start,
                                               'End':end}, 
                                              index=[row_index])
                    summary_df = pd.concat([summary_df, new_row_df])
                    summary_df.loc[row_index, 'Meter Rank']=meter_status #ADDED 12/28/2023

                    #This is gonna fail if level from pipe, must update
                    summary_df.loc[row_index, 'Node Bottom Level (m)'] = node_df.loc[node, 'invertlevel'] #ADDED 12/29/2023
                    if pd.isna(gauges.loc[gauge,'Pipe']) or pipe not in pipe_df.index:
                        summary_df.loc[row_index, 'Diameter/Height (m)'] = np.nan
                        summary_df.loc[row_index, 'Pipe Shape'] = np.nan
                    else:
                        summary_df.loc[row_index, 'Diameter/Height (m)'] = pipe_df.loc[pipe, 'pipe_height'] #ADDED 12/29/2023
                        summary_df.loc[row_index, 'Pipe Shape'] = gauges.loc[gauge,'Pipe Shape'] #ADDED 12/29/2023
                        #This is gonna fail if level from pipe, must update
                        endnames = [['uplevel','fromnodeid'],['dwlevel','tonodeid']]
                        pipe_level_found = False
                        for endname in endnames:
                            if node == pipe_df.loc[pipe, endname[1]]:
                                if pd.isna(pipe_df.loc[pipe, endname[0]]):
                                    pipe_level = node_df.loc[node, 'invertlevel']
                                else:
                                    pipe_level = pipe_df.loc[pipe, endname[0]]
                                pipe_level_found = True
                                summary_df.loc[row_index, 'Pipe Bottom Level (m)'] = pipe_level
                        if pipe_level_found:
                            pipe_top = pipe_level + summary_df.loc[row_index, 'Diameter/Height (m)']
                            summary_df.loc[row_index, 'Pipe Top Level (m)'] = pipe_top                            
                        else:
                            print('Pipe level could not be determined.')                            
                      
                    summary_df.loc[row_index, 'Result File'] = result_file #ADDED 12/29/2023

                    if type(start)==pd._libs.tslibs.timestamps.Timestamp:
                        y_minor_shift = -i * map_point_spacing
                        y_final = y + y_shift + y_minor_shift

                        color_values = []
                        for j, result_type in enumerate(result_types):
                            x_minor_shift = j * map_point_spacing
                            x_final = x + x_shift + x_minor_shift
                            threshold_column = 'Threshold ' + result_type
                            bad_data = False
                            if result_type == 'HGL':
                                result_type_csv = 'Max HGL'
                                qa = row[event_category_short + ' WL QA']     
                                summary_df.loc[row_index, 'WL QA']=qa #ADDED 12/29/2023
                                if qa.lower() == 'bad data':
                                    bad_data = True
                                else:

                                    summary_df.loc[row_index, 'Node']=node #reuse this line whenever find something that belongs in spreadsheet
                                    max_model = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)].Level.max()
                                    summary_df.loc[row_index, 'Model Peak HGL (m)']=max_model #ADDED 12/28/2023
                                    max_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Level.max()
                                    summary_df.loc[row_index, 'Measured Peak HGL (m)']=max_measured #ADDED 12/28/2023
                                    if pipe_level_found:
                                        summary_df.loc[row_index, 'Model Pipe Filled'] = max_model >= pipe_top
                                    
                                    if not math.isnan(max_measured) and max_measured != 0:
                                        diff = abs(max_model - max_measured) * 100 #cm
                                        summary_df.loc[row_index, 'Peak Difference HGL (m)']=diff/100 #ADDED 12/28/2023
                                        if pipe_level_found:
                                            summary_df.loc[row_index, 'Measured Pipe Filled'] = max_measured >= pipe_top
                                        
                                    else:
                                        diff = 'NA'

                            elif result_type == 'Flow':
                                result_type_csv = 'Max Flow'
                                qa = row[event_category_short + ' Flow QA']
                                summary_df.loc[row_index, 'Flow QA']=qa #ADDED 12/29/2023
                                if qa.lower() == 'bad data':
                                    bad_data = True
                                else:
                                    pipe = str(gauges.loc[gauge,'Pipe'])
                                    filter_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].copy()
                                    ts_seconds = (filter_model.index.max() - filter_model.index.min()).total_seconds() / (len(filter_model.index)-1)
                                    ts_per_one_hour = int(60/ts_seconds)
                                    #Remove overlaps
                                    if len(filter_model.ResultFile.unique()) > 1:
                                        row_counts = filter_model['ResultFile'].value_counts()
                                        max_resultfile = row_counts.idxmax()
                                        filter_model = filter_model[filter_model['ResultFile'] == max_resultfile]
                                    if len(filter_model) == 0:
                                        print('WARNING! ' + gauge + ' has no model records between ' + str(start) + ' and ' + str(end))                   
                                    filter_model['Hourly'] = filter_model.Discharge.rolling('1h').mean()
                                    filter_model = filter_model.iloc[ts_per_one_hour: , :] #Remove first hour to not get skewed hourly average there                        
                                    max_model = filter_model.Hourly.max()
                                    summary_df.loc[row_index, 'Model Peak Hourly Average Flow (L/s)']=max_model #ADDED 12/28/2023

                                    filter_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].copy()
                                    try:
                                        filter_measured['Hourly'] = filter_measured.Flow.rolling('1h').mean()
                                    except:
                                        filter_measured['Hourly'] = np.nan                           
                                    
                                    filter_measured = filter_measured.iloc[ts_per_one_hour: , :] #Remove first hour to not get skewed hourly average there 
                                    max_measured = filter_measured.Hourly.max()
                                    print(max_measured)
                                    summary_df.loc[row_index, 'Measured Peak Hourly Average Flow (L/s)']=max_measured #ADDED 12/28/2023
                                    if not math.isnan(max_measured) and max_measured != 0:
                                        diff_q = max_model - max_measured
                                        diff = abs(diff_q) / max_measured
                                        summary_df.loc[row_index, 'Peak Difference Flow (L/s)']=diff_q
                                        summary_df.loc[row_index, 'Peak Difference Flow (%)']=diff
                                    else:
                                        diff = 'NA'
                            else:
                                result_type_csv = 'Acc Volume'
                                qa = row[event_category_short + ' Flow QA']
                                if qa.lower() == 'bad data':
                                    bad_data = True
                                else:

                                    summary_df.loc[row_index, 'Pipe']=pipe #ADDED 12/28/2023
                                    vol_model = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)].Volume.sum()
                                    summary_df.loc[row_index, 'Model Volume (m3)']=vol_model #ADDED 12/28/2023
                                    
                                    if not math.isnan(max_measured) and max_measured != 0:
                                        vol_measured = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)].Volume.sum()
                                        summary_df.loc[row_index, 'Measured Volume (m3)']=vol_measured #ADDED 12/28/2023
                                        diff_vol = abs(vol_model - vol_measured)
                                        diff = diff_vol/vol_measured
                                        summary_df.loc[row_index, 'Volume Difference (m3)']=diff_vol #ADDED 12/28/2023
                                        summary_df.loc[row_index, 'Volume Difference (%)']=diff #ADDED 12/28/2023

                            status_column = result_type + ' Status'
                            status_value_column = status_column + ' Value'
                            if bad_data == True:
                                color = 'Black'
                                color_value = 0
                            elif diff == 'NA':
                                color = 'Grey'
                                color_value = 0
                            else:
                                if event_category[:3] == 'WWF':
                                    threshold_category = event_category[:3] + ' ' + catchment_type
                                    status_lookup_column = threshold_category + ' ' + result_type + ' Status'
                                else:
                                    threshold_category = 'DWF'
                                    status_lookup_column = event_category[:3] + ' ' + result_type + ' Status'
                                
                                for k in range(3):
                                    threshold = thresholds.loc[k,status_lookup_column]
                                    if diff >= threshold:
                                        color = thresholds.loc[k,'Color']
                                        #summary_df.loc[row_index, 'HGL Status']=color #ADDED 12/28/2023 ?????
                                        color_value = thresholds.loc[k,'index']                                    
                                        break
                                        
                                print(f'{status_lookup_column}, {threshold_category}, Diff: {diff}, {color}')
                                    
                            summary_df.loc[row_index, 'Threshold Category']=threshold_category #ADDED 12/28/2023
                            summary_df.loc[row_index, status_column] = color
                            summary_df.loc[row_index, status_value_column] = color_value
                            
                                

                            color_values.append(color_value)

                            label = ''
                            if i == 0 and j == 0:
                                label = gauge

                            result_points.append([gauge,x_final,y_final,label,event_category,result_type_csv,color])
                            summary_df.loc[row_index,result_type + ' Status'] = color
                        summary_df.loc[row_index,'Flow Volume Average Status Value'] = (summary_df.loc[row_index,'Flow Status Value'] + summary_df.loc[row_index,'Volume Status Value']) / 2
                        summary_df.loc[row_index,'Weighted Status Value'] = (summary_df.loc[row_index,'HGL Status Value'] + summary_df.loc[row_index,'Flow Volume Average Status Value']) / 2
                        
                        
                        if meter_status == 'Primary':
                            weighting_components = []
                            weighted_color_value = 0
                            value_count = 0
                            qa = 'OK'
                            for k, color_value in enumerate(color_values):
                                if color_value > 0:
                                    if k == 0:
                                        weighting_components.append(color_value) #Level is weighted as 50%
                                        value_count += 1
                                    else:
                                        weighting_components.append(color_value/2) #Flow is weighted as 25%
                                        value_count += 0.5
                                else:
                                    if k == 0:
                                        qa = 'Level Issue'
                                    else:
                                        qa = 'Flow Issue'                       

                            if len(weighting_components) == 0:
                                weighted_color_value = 0
                                weighted_color = ''
                                qa = 'All Data Issue'
                            else:
                                sum_color_values = 0
                                for weighting_component in weighting_components:
                                    sum_color_values += weighting_component
                                weighted_color_value = sum_color_values / value_count
                                weighted_color_value = round((weighted_color_value+0.001)*2,0)/2 #2.25 should round up to 2.5 as in Excel
                                weighted_color = color_dict[weighted_color_value]
                                summary_df.loc[row_index, 'Weighted Status']=weighted_color #ADDED 12/28/2023 
                            zone_key = zone + ' ' + event_category + ' Weighted'
                            zone_review.append([zone_key,'Weighted',event_category,zone,weighted_color,qa])

        
        result_points = pd.DataFrame(result_points,columns=['Gauge','X','Y','Label','SimType','ResultType','Color'])
        for index, row in result_points.iterrows():
            zone = zone_dict[row['Gauge']][0]
            meter_status = zone_dict[row['Gauge']][1]
            zone_key = zone + ' ' + row['SimType'] + ' ' + row['ResultType']
            if row['Color'] in ['Black','Grey']:
                qa = 'Level Issue' if row['SimType'] == 'Max HGL' else 'Flow Issue'
            else:
                qa = 'OK'
            if meter_status == 'Primary':
                zone_review.append([zone_key,row['ResultType'],row['SimType'],zone,row['Color'],qa])
            
        result_points.to_csv(output_folder + '\\ResultPoints.csv',index=False)
        summary_df.to_excel(output_folder + '\\Calibration_Review.xlsx')
        zone_review = pd.DataFrame(zone_review,columns=['Zone_Key','Stat','Event','Location','Status','QA'])
        zone_review.to_csv(output_folder + '\\Zone_Review.csv',index=False)

except Exception as e: 
    error_message = str(e)
    MessageBox(None, b'An error happened in permanent cell 8\n\n' + error_message.encode('utf-8'), b"Error", 0)
    raise ValueError("Error")

In [ ]:
#PERMANENT CELL 9
MessageBox(None,b'All cells ran successfully.', b'Done', 0)